# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../../')
from panav.environment.env import Room

from panav.PBS.PBS import PBS
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 


from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for ompl::msg::LogLevel already registered; second conversion method ignored.


In [2]:
#### 100 agent configuration ####
bloating_r = 0.5
vmax = 1.0


# Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }

In [3]:
from os import listdir
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]

env_names = [get_env_name(fn) for fn in listdir('./envs')]

print(env_names)

['MultiTunnel', 'Room', 'SingleTunnel', 'Warehouse']


In [5]:
from panav.environment.utils import reduced_agents_env
import signal

def timeout_handler(signum,frame):
    print("Algorithm timeout")
    raise TimeoutError("Timeout")


def run_experiments(env_name,TIMEOUT = 100):
    with open(f'./envs/{env_name}.pkl','rb') as fp:
        env_base = pkl.load(fp)
        
    max_n = len(env_base.starts)

    results = []
    timeouted = {alg:False for alg in algs}
    for N in range(10,max_n+1,10):
        env = reduced_agents_env(env_base,N)
        for name,alg in algs.items():

            if timeouted[name]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['plan'] = plan
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            
            if t>TIMEOUT:
                timeouted[name] = True
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                print("On time for algorithm", name, "N agent = ", N)


        # Save checkpoint data
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump(results,fp)



In [6]:
run_experiments("MultiTunnel")

Env: MultiTunnel Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube N agent =  10
Env: MultiTunnel Alg: Seq_Tube_nonLazy N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube_nonLazy N agent =  10
Env: MultiTunnel Alg: Seq_S2M2 N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
On time for algorithm Seq_S2M2 N agent =  10
Env: MultiTunnel Alg: ST-RRT* N agent: 10
Planning for agent 0/10
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Info:    SpaceTimeRRT: Created 76 st

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/10Info:    SpaceTimeRRT: Created 85 states (34 start + 51 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 3/10
Info:    SpaceTimeRRT: Created 90 states (29 start + 61 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

Planning for agent 18/20Info:    SpaceTimeRRT: Created 40 states (18 start + 22 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 19/20Info:    SpaceTimeRRT: Created 91 states (40 start + 51 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 134 states (60 start + 74 goal)
 ST-RRT* N agent =  20
Env: MultiTunnel Alg: Seq_Tube N agent: 30
Planning for agent 0/30
Planning for agent 1/30
Planning for agent 2/30
Planning for agent 3/30
Planning for agent 4/30
Planning for agent 5/30
Planning for agent 6/30
Planning for agent 7/30
